In [1]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
from torch.utils.data import DataLoader
from pathlib import Path
import pandas as pd
import torch
from torch.utils.data import DataLoader
from miditok import REMI
from miditok.pytorch_data import DatasetMIDI, DataCollator
from pathlib import Path
import json
import pandas as pd
import numpy as np
import random
from accelerate import Accelerator

# Define a custom DatasetMIDI subclass
class CustomDatasetMIDI(DatasetMIDI):
    def __init__(self, files_paths, labels, tokenizer, max_seq_len, bos_token_id, eos_token_id):
        super().__init__(files_paths, tokenizer, max_seq_len, bos_token_id, eos_token_id)
        self.labels = labels

    def __getitem__(self, idx):
        item = super().__getitem__(idx)
        item["labels"] = self.labels[idx]  # Add labels to the item dictionary
        return item
        
def create_data_loader(piano_scores_df):
  
  midi_paths = []
  labels=[]
  paths=piano_scores_df['piano_scores_paths']
  paths=paths.apply(lambda x: x.replace('../author-profiling-in-symbolic-music/','../../'))

  for i,score in enumerate(paths):

      midi_paths.append(Path(score))

      integer_label= 0 if piano_scores_df['composer_gender'][i] == 'Male' else 1

      labels.append(torch.tensor(integer_label))  # Modify this line to extract the label

  # Initialize the tokenizer
  tokenizer = REMI.from_pretrained("Natooz/Maestro-REMI-bpe20k")


  # Initialize the dataset
  dataset = CustomDatasetMIDI(
      files_paths=midi_paths,
      labels=labels,  # Pass the labels to the dataset
      tokenizer=tokenizer,
      max_seq_len=1024,
      bos_token_id=tokenizer.pad_token_id,
      eos_token_id=tokenizer["BOS_None"]
  )

  # Initialize the collator
  collator = DataCollator(tokenizer.pad_token_id)

  data_loader = DataLoader(dataset=dataset, collate_fn=collator, batch_size=5)

  return data_loader  

In [2]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
from torch.utils.data import DataLoader
from pathlib import Path
import pandas as pd
from tqdm import tqdm  # Import tqdm for the progress bar

# Assuming the CustomDatasetMIDI, create_data_loader, and DataCollator are defined as you provided

# Load the trained model
model = AutoModelForSequenceClassification.from_pretrained('model_e1')

# Load the custom tokenizer
tokenizer = REMI.from_pretrained("Natooz/Maestro-REMI-bpe20k")

# Create data loaders for validation and training data
test_set=pd.read_csv('../../dataframes/test_set_1.csv')

# Function to make predictions
def make_predictions(model, dataloader):
    model.eval()  # Set the model to evaluation mode
    all_predictions = []
    all_labels = []

    with torch.no_grad():  # Disable gradient calculation
        for batch in tqdm(dataloader, desc="Predicting"):
            inputs = batch['input_ids'].to(model.device)
            labels = batch['labels'].to(model.device)
            
            outputs = model(input_ids=inputs)
            logits = outputs.logits
            predictions = torch.argmax(logits, dim=-1)

            all_predictions.extend(predictions.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    return all_predictions, all_labels
    
test_set=pd.read_csv('../../dataframes/test_set_1.csv')
eval_dataloader=create_data_loader(test_set)
# Make predictions on the validation set
predictions, true_labels = make_predictions(model, eval_dataloader)

# Evaluate the predictions
accuracy = (np.array(predictions) == np.array(true_labels)).mean()
print(f"Test Accuracy: {accuracy:.4f}")

/mnt/nfs_share_magnet1/lafuente/miniconda3/envs/symbolic_music/lib/python3.12/site-packages/miditok/midi_tokenizer.py:3252: UserWarning: The special token PAD_None is present twice in your configuration. Skipping its duplicated occurrence.
  self.config = TokenizerConfig()
/mnt/nfs_share_magnet1/lafuente/miniconda3/envs/symbolic_music/lib/python3.12/site-packages/miditok/classes.py:702: UserWarning: The special token PAD_None is present twice in your configuration. Skipping its duplicated occurrence.
  return cls(**input_dict, **kwargs)
/mnt/nfs_share_magnet1/lafuente/miniconda3/envs/symbolic_music/lib/python3.12/site-packages/miditok/classes.py:702: UserWarning: Argument nb_tempos has been renamed num_tempos, you should consider to updateyour code with this new argument name.
  return cls(**input_dict, **kwargs)
/mnt/nfs_share_magnet1/lafuente/miniconda3/envs/symbolic_music/lib/python3.12/site-packages/miditok/midi_tokenizer.py:3252: UserWarning: The special token PAD_None is present 

Test Accuracy: 0.5224


In [3]:
predictions

[0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [4]:
true_labels

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
